# Retrieval Augmented Question & Answering with Amazon Bedrock using LangChain & Amazon OpenSearch

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

### Context
Previously we saw that the model told us how to to change the tire, however we had to manually provide it with the relevant data and provide the contex ourselves. We explored the approach to leverage the model availabe under Bedrock and ask questions based on it's knowledge learned during training as well as providing manual context. While that approach works with short documents or single-ton applications, it fails to scale to enterprise level question answering where there could be large enterprise documents which cannot all be fit into the prompt sent to the model. 

### Pattern
We can improve upon this process by implementing an architecure called Retreival Augmented Generation (RAG). RAG retrieves data from outside the language model (non-parametric) and augments the prompts by adding the relevant retrieved data in context. 

In this notebook we explain how to approach the pattern of Question Answering to find and leverage the documents to provide answers to the user questions.

### Challenges
- How to manage large document(s) that exceed the token limit
- How to find the document(s) relevant to the question being asked

### Proposal
To the above challenges, this notebook proposes the following strategy
#### Prepare documents
![Embeddings](./images/Embeddings_lang.png)

Before being able to answer the questions, the documents must be processed and a stored in a document store index
- Load the documents
- Process and split them into smaller chunks
- Create a numerical vector representation of each chunk using Amazon Bedrock Titan Embeddings model
- Create an index using the chunks and the corresponding embeddings
#### Ask question
![Question](./images/Chatbot_lang.png)

When the documents index is prepared, you are ready to ask the questions and relevant documents will be fetched based on the question being asked. Following steps will be executed.
- Create an embedding of the input question
- Compare the question embedding with the embeddings in the index
- Fetch the (top N) relevant document chunks
- Add those chunks as part of the context in the prompt
- Send the prompt to the model under Amazon Bedrock
- Get the contextual answer based on the documents retrieved

## Usecase
#### Dataset
To explain this architecture pattern we are using the documents from IRS. These documents explain topics such as:
- Original Issue Discount (OID) Instruments
- Reporting Cash Payments of Over $10,000 to IRS
- Employer's Tax Guide

#### Persona
Let's assume a persona of a layman who doesn't have an understanding of how IRS works and if some actions have implications or not.

The model will try to answer from the documents in easy language.


## Implementation
In order to follow the RAG approach this notebook is using the LangChain framework where it has integrations with different services and tools that allow efficient building of patterns such as RAG. We will be using the following tools:

- **LLM (Large Language Model)**: Anthropic Claude V2 available through Amazon Bedrock

  This model will be used to understand the document chunks and provide an answer in human friendly manner.
- **Embeddings Model**: Amazon Titan Embeddings available through Amazon Bedrock

  This model will be used to generate a numerical representation of the textual documents
- **Document Loader**: PDF Loader available through LangChain

  This is the loader that can load the documents from a source, for the sake of this notebook we are loading the sample files from a local path. This could easily be replaced with a loader to load documents from enterprise internal systems.

- **Vector Store**: OpenSearch available through LangChain

  In this notebook we are using Amazon OpenSearch as a vector-store to store both the embeddings and the documents. 
- **Index**: VectorIndex

  The index helps to compare the input embedding and the document embeddings to find relevant document
- **Wrapper**: wraps index, vector store, embeddings model and the LLM to abstract away the logic from the user.

## Setup

Before running the rest of this notebook, you'll need to run the cells below to (ensure necessary libraries are installed and) connect to Bedrock.

For more details on how the setup works and ⚠️ **whether you might need to make any changes**, refer to the [Bedrock boto3 setup notebook](../00_Intro/bedrock_boto3_setup.ipynb) notebook.

In this notebook, we'll also need some extra dependencies:

- [OpenSearch Python Client](https://pypi.org/project/opensearch-py/), to store vector embeddings
- [PyPDF](https://pypi.org/project/pypdf/), for handling PDF files

In [4]:
%pip install --no-build-isolation --force-reinstall --quiet\
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

Note: you may need to restart the kernel to use updated packages.


In [5]:
# %pip install -U --quiet \
#     opensearch-py==2.3.1 langchain==0.0.309 "pypdf>=3.8,<4" \
#     apache-beam \
#     datasets \
#     tiktoken

In [6]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."

os.environ["AWS_DEFAULT_REGION"] = "us-east-1" 

boto3_bedrock = bedrock.get_bedrock_client(
    #assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)


Create new client
  Using region: us-east-1


boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)


## Configure langchain

We begin with instantiating the LLM and the Embeddings model. Here we are using Anthropic Claude for text generation and Amazon Titan for text embedding.

Note: It is possible to choose other models available with Bedrock. You can replace the `model_id` as follows to change the model.

`llm = Bedrock(model_id="amazon.titan-tg1-large")`

Available model IDs include:

- `ai21.j2-ultra-v1`
- `ai21.j2-mid-v1`
- `amazon.titan-embed-text-v1`
- `amazon.titan-text-express-v1`
- `anthropic.claude-v1`
- `anthropic.claude-v2`
- `anthropic.claude-instant-v1`

In [8]:
# We will be using the Titan Embeddings Model to generate our Embeddings.
from langchain.embeddings import BedrockEmbeddings
from langchain.llms.bedrock import Bedrock
from langchain.load.dump import dumps

# - create the Anthropic Model
llm = Bedrock(
    model_id="anthropic.claude-v2", client=boto3_bedrock, model_kwargs={"max_tokens_to_sample": 200}
)
bedrock_embeddings = BedrockEmbeddings(client=boto3_bedrock)

## Data Preparation
Let's first download some of the files to build our document store. For this example we will be using public IRS documents from [here](https://www.irs.gov/publications).

In [9]:
# from urllib.request import urlretrieve

# os.makedirs("data", exist_ok=True)
# files = [
#     "https://www.irs.gov/pub/irs-pdf/p1544.pdf",
#     "https://www.irs.gov/pub/irs-pdf/p15.pdf",
#     "https://www.irs.gov/pub/irs-pdf/p1212.pdf",
# ]
# for url in files:
#     file_path = os.path.join("data", url.rpartition("/")[2])
#     urlretrieve(url, file_path)

After downloading we can load the documents with the help of [DirectoryLoader from PyPDF available under LangChain](https://python.langchain.com/en/latest/reference/modules/document_loaders.html) and splitting them into smaller chunks.

Note: The retrieved document/text should be large enough to contain enough information to answer a question; but small enough to fit into the LLM prompt. Also the embeddings model has a limit of the length of input tokens limited to 8k tokens, which roughly translates to ~32000 characters. For the sake of this use-case we are creating chunks of roughly 2000 characters with an overlap of 200 characters using [RecursiveCharacterTextSplitter](https://python.langchain.com/en/latest/modules/indexes/text_splitters/examples/recursive_text_splitter.html).

In [10]:
# !pip install pypdf

In [11]:
import numpy as np
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, PyPDFDirectoryLoader

loader = PyPDFDirectoryLoader("./data/")

documents = loader.load()
# - in our testing Character split works better with this PDF data set
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=2000,
    chunk_overlap=200,
)
docs = text_splitter.split_documents(documents)

In [12]:
avg_doc_length = lambda documents: sum([len(doc.page_content) for doc in documents]) // len(
    documents
)
avg_char_count_pre = avg_doc_length(documents)
avg_char_count_post = avg_doc_length(docs)
print(f"Average length among {len(documents)} documents loaded is {avg_char_count_pre} characters.")
print(f"After the split we have {len(docs)} documents more than the original {len(documents)}.")
print(
    f"Average length among {len(docs)} documents (after split) is {avg_char_count_post} characters."
)

Average length among 111 documents loaded is 4500 characters.
After the split we have 329 documents more than the original 111.
Average length among 329 documents (after split) is 1635 characters.


In [13]:
try:
    
    sample_embedding = np.array(bedrock_embeddings.embed_query(docs[0].page_content))
    modelId = bedrock_embeddings.model_id
    print("Embedding model Id :", modelId)
    print("Sample embedding of a document chunk: ", sample_embedding)
    print("Size of the embedding: ", sample_embedding.shape)

except ValueError as error:
    if  "AccessDeniedException" in str(error):
        print(f"\x1b[41m{error}\
        \nTo troubeshoot this issue please refer to the following resources.\
         \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
         \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")      
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error

Embedding model Id : amazon.titan-embed-text-v1
Sample embedding of a document chunk:  [ 0.3359375  -0.07910156 -0.33203125 ...  0.08837891 -0.29882812
 -0.23535156]
Size of the embedding:  (1536,)


Following the similar pattern embeddings could be generated for the entire corpus and stored in a vector store.

Firt of all we have to create a vector store. In this workshop we will use ***Amazon OpenSerach serverless.***

Amazon OpenSearch Serverless is a serverless option in Amazon OpenSearch Service. As a developer, you can use OpenSearch Serverless to run petabyte-scale workloads without configuring, managing, and scaling OpenSearch clusters. You get the same interactive millisecond response times as OpenSearch Service with the simplicity of a serverless environment. Pay only for what you use by automatically scaling resources to provide the right amount of capacity for your application—without impacting data ingestion. 

In [31]:
import boto3
import json
import os
import time

vector_store_name = 'bedrock-workshop-rag'
index_name = "bedrock-workshop-rag-index"
encryption_policy_name = "bedrock-workshop-rag-sp"
network_policy_name = "bedrock-workshop-rag-np"
access_policy_name = 'bedrock-workshop-rag-ap'
identity = boto3.client('sts').get_caller_identity()['Arn']

aoss_client = boto3.client('opensearchserverless')

# Function to create a security policy if it doesn't exist
def create_security_policy_if_not_exists(policy_name, policy_type, policy):
    try:
        aoss_client.create_security_policy(name=policy_name, policy=json.dumps(policy), type=policy_type)
        print(f"Created {policy_type} policy: {policy_name}")
    except aoss_client.exceptions.ConflictException:
        print(f"{policy_type} policy already exists: {policy_name}")

# Encryption policy
encryption_policy = {
    'Rules': [{'Resource': ['collection/' + vector_store_name], 'ResourceType': 'collection'}],
    'AWSOwnedKey': True
}
create_security_policy_if_not_exists(encryption_policy_name, 'encryption', encryption_policy)

# Network policy
network_policy = [
    {'Rules': [{'Resource': ['collection/' + vector_store_name], 'ResourceType': 'collection'}], 'AllowFromPublic': True}
]
create_security_policy_if_not_exists(network_policy_name, 'network', network_policy)

# Function to check if collection exists
def collection_exists(collection_name):
    try:
        response = aoss_client.list_collections()
        for collection_summary in response['collectionSummaries']:
            if collection_summary['name'] == collection_name:
                return True
        return False
    except Exception as e:
        print(f"Error checking collection existence: {str(e)}")
        return False

# Create collection if it doesn't exist
if not collection_exists(vector_store_name):
    collection = aoss_client.create_collection(name=vector_store_name, type='VECTORSEARCH')
    print(f"Created collection: {vector_store_name}")
else:
    print(f"Collection already exists: {vector_store_name}")

# Wait for collection to be active or failed
while collection_exists(vector_store_name):
    status = aoss_client.list_collections(collectionFilters={'name': vector_store_name})['collectionSummaries'][0]['status']
    if status in ('ACTIVE', 'FAILED'): break
    time.sleep(10)

# Access policy
access_policy = [
    {
        'Rules': [
            {
                'Resource': ['collection/' + vector_store_name],
                'Permission': ['aoss:CreateCollectionItems', 'aoss:DeleteCollectionItems', 'aoss:UpdateCollectionItems', 'aoss:DescribeCollectionItems'],
                'ResourceType': 'collection'
            },
            {
                'Resource': ['index/' + vector_store_name + '/*'],
                'Permission': ['aoss:CreateIndex', 'aoss:DeleteIndex', 'aoss:UpdateIndex', 'aoss:DescribeIndex', 'aoss:ReadDocument', 'aoss:WriteDocument'],
                'ResourceType': 'index'
            }
        ],
        'Principal': [identity],
        'Description': 'Easy data policy'
    }
]

try:
    aoss_client.create_access_policy(name=access_policy_name, policy=json.dumps(access_policy), type='data')
    print(f"Created data policy: {access_policy_name}")
except aoss_client.exceptions.ConflictException:
    print(f"Data policy already exists: {access_policy_name}")

# Function to get the collection ID
def get_collection_id(collection_name):
    try:
        response = aoss_client.list_collections()
        for collection in response['collectionSummaries']:
            if collection['name'] == collection_name:
                return collection.get('id')
        return None
    except Exception as e:
        print(f"Error retrieving collection ID: {str(e)}")
        return None

# Get host information
region = os.environ.get("AWS_DEFAULT_REGION", None)
collection_id = get_collection_id(vector_store_name)

if region and collection_id:
    host = f"{collection_id}.{region}.aoss.amazonaws.com:443"
    print(f"Host information: {host}")
else:
    print("AWS_DEFAULT_REGION is not set or unable to retrieve collection ID.")


encryption policy already exists: bedrock-workshop-rag-sp
network policy already exists: bedrock-workshop-rag-np
Collection already exists: bedrock-workshop-rag
Data policy already exists: bedrock-workshop-rag-ap
Host information: an3r7w8yqawjh1wxjzm5.us-east-1.aoss.amazonaws.com:443


In [17]:
# import boto3
# import time
# vector_store_name = 'bedrock-workshop-rag'
# index_name = "bedrock-workshop-rag-index"
# encryption_policy_name = "bedrock-workshop-rag-sp"
# network_policy_name = "bedrock-workshop-rag-np"
# access_policy_name = 'bedrock-workshop-rag-ap'
# identity = boto3.client('sts').get_caller_identity()['Arn']

# aoss_client = boto3.client('opensearchserverless')

# security_policy = aoss_client.create_security_policy(
#     name = encryption_policy_name,
#     policy = json.dumps(
#         {
#             'Rules': [{'Resource': ['collection/' + vector_store_name],
#             'ResourceType': 'collection'}],
#             'AWSOwnedKey': True
#         }),
#     type = 'encryption'
# )

# network_policy = aoss_client.create_security_policy(
#     name = network_policy_name,
#     policy = json.dumps(
#         [
#             {'Rules': [{'Resource': ['collection/' + vector_store_name],
#             'ResourceType': 'collection'}],
#             'AllowFromPublic': True}
#         ]),
#     type = 'network'
# )

# collection = aoss_client.create_collection(name=vector_store_name,type='VECTORSEARCH')

# while True:
#     status = aoss_client.list_collections(collectionFilters={'name':vector_store_name})['collectionSummaries'][0]['status']
#     if status in ('ACTIVE', 'FAILED'): break
#     time.sleep(10)

# access_policy = aoss_client.create_access_policy(
#     name = access_policy_name,
#     policy = json.dumps(
#         [
#             {
#                 'Rules': [
#                     {
#                         'Resource': ['collection/' + vector_store_name],
#                         'Permission': [
#                             'aoss:CreateCollectionItems',
#                             'aoss:DeleteCollectionItems',
#                             'aoss:UpdateCollectionItems',
#                             'aoss:DescribeCollectionItems'],
#                         'ResourceType': 'collection'
#                     },
#                     {
#                         'Resource': ['index/' + vector_store_name + '/*'],
#                         'Permission': [
#                             'aoss:CreateIndex',
#                             'aoss:DeleteIndex',
#                             'aoss:UpdateIndex',
#                             'aoss:DescribeIndex',
#                             'aoss:ReadDocument',
#                             'aoss:WriteDocument'],
#                         'ResourceType': 'index'
#                     }],
#                 'Principal': [identity],
#                 'Description': 'Easy data policy'}
#         ]),
#     type = 'data'
# )

# host = collection['createCollectionDetail']['id'] + '.' + os.environ.get("AWS_DEFAULT_REGION", None) + '.aoss.amazonaws.com:443'

Now we are ready to inject our documents into vector store. This can be easily done using [OpenSearch](https://python.langchain.com/docs/integrations/vectorstores/opensearch) implementation inside [LangChain](https://python.langchain.com/en/latest/modules/indexes/vectorstores/examples/faiss.html) which takes  input the embeddings model and the documents to create the entire vector store. Using the Index Wrapper we can abstract away most of the heavy lifting such as creating the prompt, getting embeddings of the query, sampling the relevant documents and calling the LLM. [VectorStoreIndexWrapper](https://python.langchain.com/en/latest/modules/indexes/getting_started.html#one-line-index-creation) helps us with that.

In [35]:
# !pip install opensearch-py

In [36]:
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth
from langchain.vectorstores import OpenSearchVectorSearch

service = 'aoss'
credentials = boto3.Session().get_credentials()
auth = AWSV4SignerAuth(credentials, os.environ.get("AWS_DEFAULT_REGION", None), service)

docsearch = OpenSearchVectorSearch.from_documents(
    docs,
    bedrock_embeddings,
    opensearch_url=host,
    http_auth=auth,
    timeout = 100,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection,
    index_name=index_name,
    engine="faiss",
)

## LangChain Vector Store and Querying

#### We can use the similarity search method to make a query and return the chunks of text without any LLM generating the response.

It takes a few seconds to make documents availible in index. If you will get an empty output in a next cell, just wait a little bit and retry. 

In [37]:
query = "Is it possible that I get sentenced to jail due to failure in filings?"

results = docsearch.similarity_search(query, k=3)  # our search query  # return 3 most relevant docs
print(dumps(results, pretty=True))

[
  {
    "lc": 1,
    "type": "constructor",
    "id": [
      "langchain",
      "schema",
      "document",
      "Document"
    ],
    "kwargs": {
      "page_content": "clerks of federal or state courts are discussed \nlater under Bail received by court clerks.\nHowever, you do not have to file Form 8300 \nif the transaction is not related to your trade or \nbusiness. For example, if you own a jewelry \nstore and sell your personal automobile for \nmore than $10,000 in cash, you would not sub-\nmit a Form 8300 for that transaction.\nTransaction defined.  A \u201ctransaction\u201d occurs \nwhen:\nGoods, services, or property are sold;\nProperty is rented;\nCash is exchanged for other cash;\nA contribution is made to a trust or escrow \naccount;\nA loan is made or repaid; or\nCash is converted to a negotiable instru-\nment, such as a check or a bond.\nPerson defined.  A \u201cperson\u201d includes an indi-\nvidual, a company, a corporation, a partnership, \nan association, a trust, 

#### All of these are relevant results, telling us that the retrieval component of our systems is functioning. The next step is adding our LLM to generatively answer our question using the information provided in these retrieved contexts.

## Generative Question Answering

In generative question-answering (GQA), we pass our question to the Claude-2 but instruct it to base the answer on the information returned from our knowledge base. We can do this in LangChain easily using the RetrievalQA chain.

In [38]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever())

#### Let’s try this with our earlier query:

In [39]:
qa.run(query)

" Based on the context provided, it does not seem likely that you would be sentenced to jail solely for failure to properly file certain tax forms, such as Form 8300. Some key points:\n\n- The passage discusses civil penalties like fines for failure to file Form 8300 or other information returns, but does not mention criminal penalties or jail time for such failures.\n\n- It notes that criminal penalties may apply if you don't comply with special bank deposit requirements for a special trust account for the U.S. Government, but this seems to refer to a situation where the IRS has required you to set up such an account after failing to pay over withheld taxes - not just for failure to file information returns.\n\n- In the section on Trust Fund Recovery Penalty, it notes that penalty is 100% of unpaid trust fund tax if those taxes are not paid over, and that criminal penalties may apply for failure to comply with special trust account deposit requirements in that situation. However, agai

We’re still not entirely protected from convincing yet false hallucinations by the model, they can happen, and it’s unlikely that we can eliminate the problem completely. However, we can do more to improve our trust in the answers provided.

An effective way of doing this is by adding citations to the response, allowing a user to see where the information is coming from. We can do this using a slightly different version of the RetrievalQA chain called RetrievalQAWithSourcesChain.

In [40]:
from langchain.chains import RetrievalQAWithSourcesChain

qa_with_sources = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever(search_kwargs={'k': 3}),return_source_documents=True)

In [41]:
print(dumps(qa_with_sources(query), pretty=True))

{
  "query": "Is it possible that I get sentenced to jail due to failure in filings?",
  "result": " Based on the context provided, it is possible to face criminal penalties including jail time for failure to properly file certain tax forms or comply with special bank deposit requirements related to withholding taxes. Specifically:\n\n- If you are required to file Form 8300 for cash transactions over $10,000 related to your trade or business, but willfully fail to do so, you may face up to 5 years in prison. \n\n- If you are required to deposit taxes withheld from employees into a special trust account for the government but don't comply, you may face criminal penalties.\n\n- The context mentions that if income, social security, or Medicare taxes that must be withheld aren't withheld or paid over, you may be personally liable for the trust fund recovery penalty, which can include jail time in criminal cases.\n\nSo in summary, yes, failing to properly file required tax forms or comply w

#### Now we have answered the question being asked but also included the source of this information being used by the LLM.

#### We’ve learned how to ground Large Language Models with source knowledge by using a vector database as our knowledge base. Using this, we can encourage accuracy in our LLM’s responses, keep source knowledge up to date, and improve trust in our system by providing citations with every answer.

We can use this embedding of the query to then fetch relevant documents.
Now our query is represented as embeddings we can do a similarity search of our query against our data store providing us with the most relevant information.

### Customisable option
In the above scenario you explored the quick and easy way to get a context-aware answer to your question. Now let's have a look at a more customizable option with the helpf of [RetrievalQA](https://python.langchain.com/en/latest/modules/chains/index_examples/vector_db_qa.html) where you can customize how the documents fetched should be added to prompt using `chain_type` parameter. Also, if you want to control how many relevant documents should be retrieved then change the `k` parameter in the cell below to see different outputs. In many scenarios you might want to know which were the source documents that the LLM used to generate the answer, you can get those documents in the output using `return_source_documents` which returns the documents that are added to the context of the LLM prompt. `RetrievalQA` also allows you to provide a custom [prompt template](https://python.langchain.com/en/latest/modules/prompts/prompt_templates/getting_started.html) which can be specific to the model.

Note: In this example we are using Anthropic Claude as the LLM under Amazon Bedrock, this particular model performs best if the inputs are provided under `Human:` and the model is requested to generate an output after `Assistant:`. In the cell below you see an example of how to control the prompt such that the LLM stays grounded and doesn't answer outside the context.

In [42]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

prompt_template = """Human: Use the following pieces of context to provide a concise answer in Italian to the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Assistant:"""

PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

qa_prompt = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT},
)
query = "Is it possible that I get sentenced to jail due to failure in filings?"
result = qa_prompt({"query": query})
print_ww(result["result"])

 Sì, è possibile che una persona venga condannata al carcere per non aver adempiuto correttamente
agli obblighi di dichiarazione fiscale. Infatti, secondo quanto riportato nel testo, in alcuni casi
specifici come il mancato versamento delle ritenute previdenziali e assistenziali dei dipendenti,
può scattare una pena detta "Trust fund recovery penalty" che prevede una multa del 100% sulle tasse
non pagate. Inoltre, se le tasse non pagate non possono essere recuperate immediatamente, chi è
stato ritenuto responsabile di tale omissione può andare incontro a conseguenze penali che includono
la reclusione, soprattutto se l'omissione è st


In [43]:
print(dumps(result, pretty=True))

{
  "query": "Is it possible that I get sentenced to jail due to failure in filings?",
  "result": " S\u00ec, \u00e8 possibile che una persona venga condannata al carcere per non aver adempiuto correttamente agli obblighi di dichiarazione fiscale. Infatti, secondo quanto riportato nel testo, in alcuni casi specifici come il mancato versamento delle ritenute previdenziali e assistenziali dei dipendenti, pu\u00f2 scattare una pena detta \"Trust fund recovery penalty\" che prevede una multa del 100% sulle tasse non pagate. Inoltre, se le tasse non pagate non possono essere recuperate immediatamente, chi \u00e8 stato ritenuto responsabile di tale omissione pu\u00f2 andare incontro a conseguenze penali che includono la reclusione, soprattutto se l'omissione \u00e8 st",
  "source_documents": [
    {
      "lc": 1,
      "type": "constructor",
      "id": [
        "langchain",
        "schema",
        "document",
        "Document"
      ],
      "kwargs": {
        "page_content": "clerks 

### Clean up
You have reached the end of this workshop. Following cell will delete all created resources.


In [ ]:
aoss_client.delete_collection(id=collection['createCollectionDetail']['id'])
aoss_client.delete_access_policy(name=access_policy_name, type='data')
aoss_client.delete_security_policy(name=encryption_policy_name, type='encryption')
aoss_client.delete_security_policy(name=network_policy_name, type='network')


## Conclusion
Congratulations on completing this moduel on retrieval augmented generation! This is an important technique that combines the power of large language models with the precision of retrieval methods. By augmenting generation with relevant retrieved examples, the responses we recieved become more coherent, consistent and grounded. You should feel proud of learning this innovative approach. I'm sure the knowledge you've gained will be very useful for building creative and engaging language generation systems. Well done!

In the above implementation of RAG based Question Answering we have explored the following concepts and how to implement them using Amazon Bedrock and it's LangChain integration.

- Loading documents and generating embeddings to create a vector store
- Retrieving documents to the question
- Preparing a prompt which goes as input to the LLM
- Present an answer in a human friendly manner
- keep source knowledge up to date, and improve trust in our system by providing citations with every answer.

### Take-aways
- Experiment with different Vector Stores
- Leverage various models available under Amazon Bedrock to see alternate outputs
- Explore options such as persistent storage of embeddings and document chunks
- Integration with enterprise data stores

# Thank You